# Anything2Img 测试

本 notebook 用于测试 `src/autocoder/common/anything2img.py` 模块的功能。

In [ ]:
# 导入必要的模块
import os
import byzerllm
from autocoder.common import AutoCoderArgs
from autocoder.common.anything2img import Anything2Img
from loguru import logger

# 初始化测试环境
logger.remove()
logger.add(lambda msg: print(msg), level="INFO")

# 创建测试输出目录
test_output_dir = "./test_output"
os.makedirs(test_output_dir, exist_ok=True)

# 初始化 AutoCoderArgs
args = AutoCoderArgs(
    output=test_output_dir,
    model="deepseek_chat",
    anti_quota_limit=0.1
)

# 初始化 LLM
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_vl")

# 初始化 Anything2Img
converter = Anything2Img(llm, args)

## 测试 PDF 转换

In [ ]:
# 测试 PDF 文件转换
pdf_file = "test.pdf"  # 请确保测试目录下有这个文件
if os.path.exists(pdf_file):
    image_paths = converter.convert_pdf(pdf_file)
    print(f"转换后的图片路径: {image_paths}")
    
    # 测试图片分析
    for img_path in image_paths:
        page_info = converter.analyze_image.with_llm(llm).with_return_type(Page).run(img_path)
        print(f"图片分析结果: {page_info}")
else:
    print(f"测试文件 {pdf_file} 不存在，跳过 PDF 测试")

## 测试 DOCX 转换

In [ ]:
# 测试 DOCX 文件转换
docx_file = "test.docx"  # 请确保测试目录下有这个文件
if os.path.exists(docx_file):
    image_paths = converter.convert_docx(docx_file)
    print(f"转换后的图片路径: {image_paths}")
    
    # 测试图片分析
    for img_path in image_paths:
        page_info = converter.analyze_image.with_llm(llm).with_return_type(Page).run(img_path)
        print(f"图片分析结果: {page_info}")
else:
    print(f"测试文件 {docx_file} 不存在，跳过 DOCX 测试")

## 测试 Markdown 生成

In [ ]:
# 测试 Markdown 生成
test_file = pdf_file if os.path.exists(pdf_file) else docx_file if os.path.exists(docx_file) else None
if test_file:
    markdown_content = converter.to_markdown(test_file)
    print("生成的 Markdown 内容:")
    print(markdown_content)
    
    # 保存 Markdown 文件
    output_md = os.path.join(test_output_dir, "output.md")
    with open(output_md, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    print(f"Markdown 文件已保存到: {output_md}")
else:
    print("没有可用的测试文件，跳过 Markdown 测试")

## 清理测试环境

In [ ]:
# 删除测试输出目录
import shutil
shutil.rmtree(test_output_dir)
print(f"已删除测试输出目录: {test_output_dir}")